In [ ]:
#import basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#check versions
print(pd.__version__)
print(np.__version__)

# Importing WordCloud for text visualization
from wordcloud import WordCloud

# Importing NLTK for natural language processing
import nltk
from nltk.corpus import stopwords # For stopwords


2.3.0
2.3.1
